In [1]:
!pip install --upgrade opencv-python

     |████████████████████████████████| 60.5 MB 1.3 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [12]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2022-04-24 17:41:32--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  30.5MB/s    in 6.3s    

2022-04-24 17:41:39 (27.4 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [13]:
!tar -xzvf lfw.tgz

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
lfw/Luis_Sanchez/
lfw/Luis_Sanchez/Luis_Sanchez_0001.jpg
lfw/Juliette_Binoche/
lfw/Juliette_Binoche/Juliette_Binoche_0001.jpg
lfw/Joel_Todd/
lfw/Joel_Todd/Joel_Todd_0001.jpg
lfw/Allan_Wagner/
lfw/Allan_Wagner/Allan_Wagner_0001.jpg
lfw/Alejandro_Fernandez/
lfw/Alejandro_Fernandez/Alejandro_Fernandez_0001.jpg
lfw/Guillaume_Depardieu/
lfw/Guillaume_Depardieu/Guillaume_Depardieu_0001.jpg
lfw/Delphine_Chuillot/
lfw/Delphine_Chuillot/Delphine_Chuillot_0001.jpg
lfw/Paul_LeClerc/
lfw/Paul_LeClerc/Paul_LeClerc_0001.jpg
lfw/Eddie_Jordan/
lfw/Eddie_Jordan/Eddie_Jordan_0001.jpg
lfw/Lionel_Richie/
lfw/Lionel_Richie/Lionel_Richie_0001.jpg
lfw/Lionel_Richie/Lionel_Richie_0002.jpg
lfw/Jose_Viegas_Filho/
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0001.jpg
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0002.jpg
lfw/Roger_Mahony/
lfw/Roger_Mahony/Roger_Mahony_0001.jpg
lfw/Clay_Aiken/
lfw/Clay_Aiken/Clay_Aiken_0001.jpg
lfw/Clay_Aiken/Clay_Aiken_0002.jpg
lfw/C

In [19]:
import os 
import glob 
import shutil
import numpy as np 
import cv2 
import tqdm

In [15]:
dataset_path = './lfw/*/'
for folder in glob.glob(dataset_path):
    file_paths = folder + '*.*' # get all file with any extension
    if len(glob.glob(file_paths)) == 1: # remove folder has 1 image
        shutil.rmtree(folder)

print('Total people:', len(glob.glob(dataset_path)))
print('Total images: ', len(glob.glob(dataset_path+'*.*')))

Total people: 1680
Total images:  9164


In [28]:
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

In [48]:
detector = cv2.FaceDetectorYN.create(
        "yunet.onnx",
        "",
        (320, 320),
        0.9,
        0.3,
        5000
    )

recognizer = cv2.FaceRecognizerSF.create(
            "face_recognizer_fast.onnx","")
detector.setInputSize((250, 250))


In [43]:
img = cv2.imread('face1.png')
# detector.setInputSize((img.shape[1], img.shape[0]))

face = detector.detect(img)
print(face)
# visualize(img, face)
cv2.imwrite('result.jpg', img)

face1_align = recognizer.alignCrop(img, face[1][0])
cv2.imwrite('img_align.jpg', face1_align)

face1_feature = recognizer.feature(face1_align)
print(face1_feature.shape)

(1, array([[183.56718  , 117.29676  ,  84.21616  ,  81.65764  , 209.05481  ,
        154.15823  , 240.47581  , 153.70473  , 225.43259  , 173.48279  ,
        211.36511  , 181.63861  , 237.2764   , 181.17538  ,   0.9996943]],
      dtype=float32))
(1, 128)


In [46]:
a = l2_normalize(face1_feature[0])
a.shape

(128,)

In [49]:
dataset_path = './lfw/*/'
emb_A = []
emb_B = []
emb_norm_A = []
emb_norm_B = []

for folder in tqdm.tqdm(glob.glob(dataset_path)):
    paths = glob.glob(folder + '*.*')[:2]
    path1 = paths[0]
    path2 = paths[1]

    img1 = cv2.imread(path1)
    img2 = cv2.imread(path2)

    # Face detect
    faces1 = detector.detect(img1)
    faces2 = detector.detect(img2)

    if faces1[1] is None or faces2[1] is None:
        continue 

    # Align faces
    face1_align = recognizer.alignCrop(img1, faces1[1][0])
    face2_align = recognizer.alignCrop(img2, faces2[1][0])

    # Extract features
    face1_feature = recognizer.feature(face1_align)[0]
    face2_feature = recognizer.feature(face2_align)[0]

    # Norm 
    face1_norm = l2_normalize(face1_feature)
    face2_norm = l2_normalize(face2_feature)

    emb_A.append(face1_feature)
    emb_B.append(face2_feature)
    emb_norm_A.append(face1_norm)
    emb_norm_B.append(face2_norm)

100%|██████████| 1680/1680 [01:55<00:00, 14.58it/s]


In [50]:
emb_A = np.array(emb_A)
emb_B = np.array(emb_B)
emb_norm_A = np.array(emb_norm_A)
emb_norm_B = np.array(emb_norm_B)

print(emb_A.shape)
print(emb_B.shape)
print(emb_norm_A.shape)
print(emb_norm_B.shape)

(1451, 128)
(1451, 128)
(1451, 128)
(1451, 128)


In [51]:
np.save('cv2_emb_A.npy', emb_A)
np.save('cv2_emb_B.npy', emb_B)
np.save('cv2_emb_norm_A.npy', emb_norm_A)
np.save('cv2_emb_norm_B.npy', emb_norm_B)

In [52]:
emb_B_T = emb_B.reshape((1451, 1, 128))

In [53]:
sub = emb_A - emb_B_T
sub.shape

(1451, 1451, 128)

In [54]:
mul = sub * sub
mul.shape

(1451, 1451, 128)

In [55]:
sum = np.sum(mul, axis=2)
sum.shape

(1451, 1451)

In [56]:
sqrt = np.sqrt(sum)
sqrt.shape

(1451, 1451)

In [57]:
import pandas as pd 

df = pd.DataFrame(sqrt)
df.head()

0          1          2          3          4          5     \
0   8.557764  15.518267  16.657316  18.327333  17.206211  17.114511   
1  16.023251  12.077537  16.939638  16.491127  16.600996  14.573878   
2  15.837371  16.049942   8.210246  17.503048  17.300104  16.708759   
3  17.267843  15.642186  17.243952  11.721290  15.952477  15.920982   
4  16.971897  15.885882  17.740295  17.411133   8.117425  16.063610   

        6          7          8          9     ...       1441       1442  \
0  17.102766  15.910304  18.009317  14.406752  ...  17.101757  16.444658   
1  17.659494  15.782229  15.546625  15.459695  ...  17.476177  16.422243   
2  15.259255  16.958010  17.554113  13.513405  ...  16.778578  16.311514   
3  17.276188  16.157778  16.397352  15.543137  ...  16.363205  16.350956   
4  18.324543  18.359221  18.912247  17.192873  ...  18.733360  18.278034   

        1443       1444       1445       1446       1447       1448  \
0  17.845625  18.699411  16.668365  13.598096  16.193844  17.744345   
1  16.143257  16.684551  16.020027  14.531737  15.604975  17.661163   
2  17.675728  19.198830  16.284479  14.368250  17.385315  15.782142   
3  15.845406  17.496853  17.515839  13.760386  17.212812  17.216473   
4  17.460024  17.271189  17.315538  15.250975  17.552208  19.306236   

        1449       1450  
0  16.023674  16.868681  
1  17.249640  16.575333  
2  17.501143  17.866262  
3  16.093681  16.701008  
4  18.347376  17.178806  

[5 rows x 1451 columns]

In [58]:
df.to_csv('cv2_euclidean_distance.csv')

In [63]:
emb_norm_A = np.load('cv2_emb_norm_A.npy')
emb_norm_B = np.load('cv2_emb_norm_B.npy')

In [64]:
emb_norm_B_T = emb_norm_B.reshape((1451, 1, 128))

In [65]:
sub = emb_norm_A - emb_norm_B_T
sub.shape

(1451, 1451, 128)

In [66]:
mul = sub * sub
mul.shape

(1451, 1451, 128)

In [67]:
sum = np.sum(mul, axis=2)
sum.shape

(1451, 1451)

In [68]:
sqrt = np.sqrt(sum)
sqrt.shape

(1451, 1451)

In [69]:
import pandas as pd 

df = pd.DataFrame(sqrt)
df.head()

0         1         2         3         4         5         6     \
0  0.679411  1.287798  1.302545  1.412704  1.359094  1.408496  1.321712   
1  1.316130  1.040880  1.369032  1.311884  1.356196  1.244370  1.409629   
2  1.277166  1.355502  0.649956  1.369038  1.387820  1.398448  1.196211   
3  1.434577  1.365026  1.408923  0.936458  1.317342  1.375890  1.393445   
4  1.295642  1.261791  1.335724  1.293622  0.613714  1.266173  1.364863   

       7         8         9     ...      1441      1442      1443      1444  \
0  1.303005  1.391380  1.258393  ...  1.290757  1.292773  1.422696  1.428021   
1  1.340784  1.239254  1.412960  ...  1.360110  1.334668  1.331952  1.313907   
2  1.412474  1.376347  1.202812  ...  1.283898  1.302016  1.431566  1.487846   
3  1.389165  1.321302  1.440640  ...  1.284673  1.343329  1.321998  1.392864   
4  1.443333  1.408207  1.438707  ...  1.365392  1.383039  1.338118  1.272160   

       1445      1446      1447      1448      1449      1450  
0  1.301163  1.240488  1.282240  1.349436  1.292351  1.309680  
1  1.291919  1.401637  1.277951  1.385646  1.441321  1.329137  
2  1.290380  1.348003  1.398164  1.216473  1.434610  1.408103  
3  1.428602  1.345863  1.425783  1.364125  1.360244  1.353507  
4  1.301639  1.325602  1.336809  1.416764  1.421740  1.284949  

[5 rows x 1451 columns]

In [70]:
df.to_csv('cv2_euclidean_norm_distance.csv')

In [11]:
!python face_detect.py -i1 face1.png -i2 face2.png

[ERROR:0@0.011] global /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp (2516) parseShape DNN/ONNX(Shape): dynamic 'zero' shapes are not supported, input 243 [ 0 0 0 51 ]
[ERROR:0@0.011] global /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp (2516) parseShape DNN/ONNX(Shape): dynamic 'zero' shapes are not supported, input 250 [ 0 0 0 34 ]
[ERROR:0@0.011] global /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp (2516) parseShape DNN/ONNX(Shape): dynamic 'zero' shapes are not supported, input 257 [ 0 0 0 34 ]
[ERROR:0@0.011] global /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp (2516) parseShape DNN/ONNX(Shape): dynamic 'zero' shapes are not supported, input 264 [ 0 0 0 51 ]
[ERROR:0@0.011] global /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp (2516) parseShape DNN/ONNX(Shape): dynamic 'zero' shapes are not supported, input 297 [ 0 -4 ]
Face 0, top-left coordinates: (184, 117), box width: 84, box height 82, score: 1.00
Face 0, top-left coordinates: (296, 99), box width: 86, box 